### 1.1Introduction and Background

To better understand this report, it is crucial to understand what an option contract is and what are its basic features. When one buys an option contract, they are purchasing a right but not an obligation to purchase (call option) or sell (put option) an asset at a certain price (strike price) by a certain date (maturity date). The purchaser of the option can choose what they prefer as the strike price and maturity date, and the price of the contract will adjust accordingly. When the maturity date of a call option hits, if the underlying asset's price is at or above the strike price (at-the-money (ATM), in-the-money (ITM)), the purchaser of the call option can choose to exercise the contract and purchase the underlying asset at the strike price but if the underlying asset's price is below the strike price (Out-of-the-money (OTM)), the contract expires worthless because it does not make sense to exercise. Similarly, when the maturity date of a put option hits, if the underlying asset's price is at or below the strike price (ATM, ITM), the purchaser of the put option can choose to exercise the contract and sell the underlying asset at the strike price, but the contract expires worthless if the asset's price is above the strike price (OTM).

The goal of this project was to create and compare the accuracy of multiple options pricing models and evaluate its accuracy based on comparisons of finite element models to a finite difference model and a theoretical solution. There were three models in total, two finite element models and one finite difference model.

The Black-Scholes partial differential equation (PDE) is an equation that is used to determine options contract pricing, which is often used in the stock market. Specifically, this PDE is most commonly used for European option contracts as they cannot be exercised prior to the maturity date, while American options can be exercised prior to maturity. Mathematical adjustments to the Black-Scholes PDE must be made to account for American options. This paper focuses on European options and neglects the existence of the option to exercise contracts early. Equation 1 below is the most common form of the Black-Scholes PDE, and it resembles a general nonsteady state advection-diffusion PDE.

$$
\begin{equation*}
\frac{\partial V}{\partial t}-r S \frac{\partial V}{\partial S}+\frac{1}{2} \sigma^{2} S^{2} \frac{\partial^{2} V}{\partial S^{2}}-r V=0 \tag{1}
\end{equation*}
$$

Where $V$ represents the value of the option contract (\$), $t$ represents the time in years since option purchase. $0 \leq t \leq T$ where $T$ is the time in years at contract maturity, $S$ represents the stock price of the asset underlying the option contract (\$), $\sigma$ represents the implied volatility of the option contract, and $r$ represents the risk-free interest rate, typically taken as the 10-year treasury bond yield as a yearly interest rate.

A few of the biggest weaknesses of the Black-Scholes PDE are that it does not account for early exercising of contracts, as explained above, and it assumes that implied volatility and risk-free interest rate are constant, which they are not. Typically, the interest rate does not fluctuate significantly, but it is not uncommon for implied volatility to greatly fluctuate from hour to hour. Therefore, in practice, the resulting finite difference or finite element model must be run periodically to update itself.

Other limitations of the Black-Scholes Model as outlined by Investopedia are that they assume continuous trading, no broker fees or bid/ask pricing spreads, it assumes that stock prices follow
a lognormal pattern (i.e. geometric Brownian motion) which ignores large price swings, it assumes no dividends are paid out, and it assumes the market is run by ideal banks, meaning there are no arbitrage opportunities due to pricing discrepancies (Seth, 2022).

The finite element (FE) method that was used was a 1D application of the Galerkin Method of Weighted Residuals (GMWR) in the price dimension, followed by a 1D application of the Crank-Nicolson Method (CNM) in the temporal dimension.


### 1.2Boundary Conditions

In order to set boundary conditions for this problem, a simple time transformation must be performed as the current form of equation 1 is a backward-looking equation. There are terminal boundary conditions that must be flipped to become initial conditions. The transformation is done using equations 2 to 4 below.

$$
\begin{gather*}
\tau=T-t  \tag{2}\\
d \tau=-d t  \tag{3}\\
-\frac{\partial V}{\partial \tau}-r S \frac{\partial V}{\partial S}+\frac{1}{2} \sigma^{2} S^{2} \frac{\partial^{2} V}{\partial S^{2}}-r V=0 \tag{4}
\end{gather*}
$$

Where $\tau$ represents the time in years until contract maturity, where $\tau=0$ is the maturity date and $\tau=T$ is the date of option contract purchase.

The boundary conditions of the Black-Scholes PDE are specified below.

For call option contracts:

$$
\begin{aligned}
& V(0, \tau)=0 \\
& V(\infty, \tau)=S-K e^{-r \tau} \\
& V(S, 0)=\max (S-K, 0)
\end{aligned}
$$

For put option contracts:

$$
\begin{aligned}
& V(0, \tau)=K e^{-r \tau} \\
& V(\infty, \tau)=0 \\
& V(S, 0)=\max (K-S, 0)
\end{aligned}
$$

Where $K$ is defined as the strike price of the option contract. The $e^{-r \tau}$ is the application of the time value of money and it represents the discounting of money in time from the maturity date to any date of interest to determine what money is worth as it moves throughout time. Discounting is the concept of decreasing future money back to the present at a given interest rate, compounding is the concept of increasing present money to the future at a given interest rate. In this case, compound interest is applied continuously, meaning interest is applied at every infinitesimal time increment.

Note that an upper bound asset price of infinity is impossible, so the upper bound was set to a small multiple of the current asset price (anywhere from $2 \mathrm{x}$ to $5 \mathrm{x}$ ). This results in a mesh that provides sufficient information as it is quite rare to more than double the value of an asset over the timeframes that stock option contracts are typically offered at. Additionally, having lower
upper bounds on asset price creates a mesh that does not require as many elements to converge because each element is smaller using the same number of elements and a uniform mesh.

This project focuses only on call option contracts for simplicity and to avoid redundancy. Put option contracts can be found with changes in boundary conditions, and no changes in any other part of the FE model.


### 1.3Global to Local Transformation

Similar to most 1D linear FE approximations, a global to local coordinate transformation simplifies the problem by homogenizing the integrations. Table 1 below summarizes the 1D linear iso-parametric transformations required to complete the FE model.

Table 1 - Summary of Iso-Parametric Global to Local Transformations

| Variable | Global Coordinates | Local Coordinates |
| :---: | :---: | :---: |
| $N_{1}^{e}$ | $\frac{\left(S_{2}^{e}-S\right)}{\Delta S}$ | $\frac{1}{2}(1-\eta)$ |
| $N_{2}^{e}$ | $\frac{\left(S-S_{1}^{e}\right)}{\Delta S}$ | $\frac{1}{2}(1+\eta)$ |
| $\frac{d N_{1}^{e}}{d \eta}$ | - | $-\frac{1}{2}$ |
| $\frac{d N_{2}^{e}}{d \eta}$ | - | $\frac{1}{2}$ |
| $\frac{d N_{1}^{e}}{d S}$ | $-\frac{1}{\Delta S}$ | - |
| $\frac{d N_{2}^{e}}{d S}$ | $\frac{1}{\Delta S}$ | - |


| Limits of Integration | $\int_{S_{1}^{e}}^{S_{2}^{e}}$ | $\int_{-1}^{1}$ |
| :---: | :---: | :---: |
| $\frac{d S}{d \eta}$ | $\frac{\Delta S}{2}$ | $\frac{\Delta S}{2}$ |

For greater detail on the global to local transformation, refer to Appendix B page 1 for the hand calculations.

### 2.1Finite Element Derivation - Advection-Diffusion PDE

Applying GMWR to equation 4 results in equation 5 below.

$$
\begin{equation*}
\int_{S_{1}^{e}}^{S_{2}^{e}}\left(-\frac{\partial V}{\partial \tau}-r S \frac{\partial V}{\partial S}+\frac{1}{2} \sigma^{2} S^{2} \frac{\partial^{2} V}{\partial S^{2}}-r V\right) N_{i} d S=0 \tag{5}
\end{equation*}
$$

Where $N_{i}$ is a row vector of the element shape functions. $V$ can be approximated by equation 6 below.

$$
\begin{equation*}
V^{\sim}(S, \tau)=\sum_{i=1}^{2} N_{i}(S) * V_{i}(\tau) \tag{6}
\end{equation*}
$$

Using integration by parts and global to local transformations, equation 5 simplifies into the form of equation 7 below.

$$
\begin{equation*}
[K] * \frac{\partial V_{j}}{\partial \tau}+[C] * V_{j}=[\alpha] \tag{7}
\end{equation*}
$$

Applying the Crank-Nicolson method to equation 7 results in equation 8 below.

$$
\begin{equation*}
[K] * \frac{V_{j}^{\tau+\Delta \tau}-V_{j}^{\tau}}{\Delta \tau}=\theta\left([\alpha]-[C] * V_{j}^{\tau+\Delta \tau}\right)+(1-\theta)\left([\alpha]-[C] * V_{j}^{\tau}\right) \tag{8}
\end{equation*}
$$

Where $\theta=\frac{1}{2}$ and $[\alpha]$ is a by-product from using integration by parts and was canceled out in the derivation. The matrices in equation 8 are defined below as:

$$
\begin{aligned}
& {[K]=\frac{\Delta S}{2} \int_{-1}^{1} N_{j} N_{i} d \eta \text {, where } N_{j} \text { is a column vector of the element shape functions. }} \\
& {[C]=\frac{\Delta S}{2}\left(\left(\sigma^{2}-r\right) \int_{-1}^{1} S \frac{\partial N_{j}}{\partial S} N_{i} d \eta+\frac{1}{2} \sigma^{2} \int_{-1}^{1} S^{2} \frac{\partial N_{j}}{\partial S} \frac{\partial N_{i}}{\partial S} d \eta+r \int_{-1}^{1} N_{j} N_{i} d \eta\right) .} \\
& {[\alpha]=\left.\frac{1}{2} \sigma^{2} S^{2} N_{i}\left(\frac{\partial V^{\sim}}{\partial S}\right)\right|_{S_{1}} ^{S_{2}} \text {. }}
\end{aligned}
$$

See Appendix B pages 1 to 3 for the hand calculations of the full derivation using GMWR and Crank-Nicolson on the classical Black-Scholes PDE.

### 2.2Transformation to Simplified Diffusion/Heat Equation

Using principles from Stochastic calculus, such as Itô's Lemma, and assuming Geometric

Brownian Motion, the Black-Scholes PDE was transformed into a simplified diffusion equation shown in equation 9 below (quantpie, 2019).

$$
\begin{equation*}
\frac{\partial F}{\partial \tau}=\frac{1}{2} \sigma^{2} \frac{\partial^{2} F}{\partial x^{2}} \tag{9}
\end{equation*}
$$

Where $F=V e^{r \tau}, x=\ln (S)+\left(r-\frac{1}{2} \sigma^{2}\right) \tau$, and the boundary conditions of the call option become:

$$
\begin{aligned}
& F(0, \tau)=0 \\
& F(\infty, \tau)=\left(e^{x-\left(r-\frac{1}{2} \sigma^{2}\right) \tau}-K e^{-r \tau}\right) * e^{r \tau} \\
& F(x, 0)=\max \left(e^{x}-K, 0\right)
\end{aligned}
$$

See Appendix B page 5 for the full derivation of the transformation to this simplified PDE form.

### 2.3Finite Element Derivation - Heat Equation

Applying GMWR to equation 9 results in equation 10 below.

$$
\begin{equation*}
\int_{S_{1}^{e}}^{S_{2}^{e}}\left(\frac{\partial F}{\partial \tau}-\frac{1}{2} \sigma^{2} \frac{\partial^{2} F}{\partial x^{2}}\right) N_{i} d x=0 \tag{10}
\end{equation*}
$$

Using the same methodology as equations 6, 7, and 8, equation 10 simplifies to equation 11 below after applying GMWR and Crank-Nicolson method.

$$
\begin{equation*}
[K] * \frac{F_{j}^{\tau+\Delta \tau}-F_{j}^{\tau}}{\Delta \tau}=\theta\left([\alpha]-[C] * F_{j}^{\tau+\Delta \tau}\right)+(1-\theta)\left([\alpha]-[C] * F_{j}^{\tau}\right) \tag{11}
\end{equation*}
$$

Where $\theta=\frac{1}{2}$ and $[\alpha]$ is a by-product from using integration by parts and was canceled out in the derivation. The matrices in equation 11 are defined below as:

$$
\begin{aligned}
& {[K]=\frac{\Delta x}{2} \int_{-1}^{1} N_{j} N_{i} d \eta, \text { where } N_{j} \text { is a column vector of the element shape functions. }} \\
& {[C]=\frac{\Delta x}{2}\left(\frac{1}{2} \sigma^{2} \int_{-1}^{1} \frac{\partial N_{j}}{\partial x} \frac{\partial N_{i}}{\partial x} d \eta\right) .} \\
& {[\alpha]=\left.\frac{1}{2} \sigma^{2} N_{i}\left(\frac{\partial F^{\sim}}{\partial x}\right)\right|_{x_{1}} ^{x_{2}} .}
\end{aligned}
$$

See Appendix B page 6 for the hand calculations of the derivation using GMWR and CrankNicolson on the simplified Black-Scholes PDE.


### 2.4Control Groups and Comparisons

To act as a control baseline comparison, finite difference approximation was applied to equation 4 using central differencing in the price dimension, and Crank-Nicolson Method in the temporal dimension. Since the derivation is near identical to Lecture 3, and the application is near identical to Assignment 3, the derivation steps will not be summarized in the body of this report. See Appendix B page 4 for the hand calculations of the finite difference derivation using the standard Black-Scholes PDE.

Additionally, there is a form of the Black-Scholes equation that simplifies into an equation form that can be solved and plotted for any given time. This was derived from the standard convolution method and is calculated using standard normal cumulative distribution functions as shown in equation 12 below (Shorif Hossan et al., 2020).

$$
\begin{equation*}
\text { Call Option Price }=S * N\left(d_{1}\right)-K e^{-r \tau} * N\left(d_{2}\right) \tag{12}
\end{equation*}
$$

Where $N()$ is defined as the standard normal cumulative distribution function, and $d_{1}$ and $d_{2}$ are defined as followed.

$$
d_{1}=\frac{\ln \left(\frac{S}{K}\right)+\left(r+\frac{\sigma^{2}}{2}\right) \tau}{\sigma \sqrt{\tau}}, d_{2}=\frac{\ln \left(\frac{S}{K}\right)+\left(r-\frac{\sigma^{2}}{2}\right) \tau}{\sigma \sqrt{\tau}}
$$

For this project, equation 12 was plotted at $\tau=T$ with every FE approximated model to evaluate the accuracy of each model. Additionally, equation 12 was solved for at the given stock price and $\tau=T$, to represent the current option price, which was then compared with the FE approximation of the current option price.

It is difficult to compare these results with real markets as the assumptions made in BlackScholes equation are ideal. In reality, markets are not as price efficient, compounding styles vary,
brokerage fees exist, arbitrage opportunities and pricing spreads exist, etc. Therefore, accuracy was determined based on comparison with equation 12.



### 3.1 Example: Apple Stock

For the first iteration of results, all models attempted to solve the call option price of Apple stock with the following given information:

- Purchase date April 5, 2022. Maturity date April 22, 2022. 17 days until expiration.
- Strike price: $\mathrm{K}=\$ 150$
- $\quad$ Stock price: $\mathrm{S}=\$ 178.44$
- Implied volatility: $\sigma=39.43 \%$
- Risk-free rate: $\mathrm{r}=2.441 \%$
- 0.17 -day time-steps (100-time steps in total)
- 100 -price steps in total
- $\quad$ Upper bound of $2 * S=\$ 356.88$


The current option price using Finite Element Method is $\$ 29.7014$
The current option price using Black-Scholes Equation is $\$ 28.7122>>$


The current option price using Finite Element Method is $\$ 28.7261$
The current option price using Black-Scholes Equation is $\$ 28.7122>>$


### 3.2 Analysis

From observation of the examples in section 3.1, it can be immediately recognized that the FE model using equation 4 (Advection-Diffusion PDE) probably has an implementation flaw either in its derivation or its MATLAB script as its pricing behavior is different than the other models and its pricing accuracy is far worse than the other models. However, the derivation of the model follows the same logic and sequences as the FE model using equation 9 (Diffusion/Heat PDE). Assuming that the implementation of the FE model using the Heat PDE is correct, the errors of the FE model using the Advection-Diffusion PDE should not be a fundamental flaw in logic and is likely caused by human errors or typos in the derivation or code. Further analysis from this point will neglect the FE model using the Advection-Diffusion PDE as it is incorrect.

One unexpected trend found in section 3.1 is that the finite difference model is the most accurate of the three models for all examples, when traditionally, the finite element method should be more accurate than the finite difference method.


### 3.3 Sanity Checks

It was considered unusual that the Heat FE model was less accurate than the finite difference model, as FE models are usually more accurate that finite difference models at the cost of higher computation time. Two procedures were completed to act as sanity checks to ensure that the Heat FE model was implemented correctly.

The first sanity check was to apply the same GMWR plus CNM to the concrete chloride concentration problem in assignment 3 to see if the resulting model would converge to the exact solution. Since the governing PDE for the Heat FE model heavily resembled the governing PDE for the concrete chloride concentration problem, the derivation and code implementation of the concrete chloride concentration problem was near identical to the Heat FE model. The only differences between the two problems were that the boundary conditions were different from each other, and the coefficient in front of the double derivative with respect to the space dimension were different. See Appendix B page 7 for the detailed derivation of the concrete chloride concentration FE model using GMWR and CNM, and Appendix A for the full MATLAB script that creates the concrete chloride concentration FE model and note its high resemblance to the derivation and MATLAB script of the Heat FE model.


Since the FE model for the concrete chloride concentration problem converged to the exact theoretical solution using GMWR and CNM, it was concluded that this model was implemented correctly, and by extension, the Heat FE model for Black-Scholes PDE was also implemented correctly.

The second sanity check was to compare my results with other scholarly articles that also applied GMWR and CNM to the Black-Scholes equation. Most other papers that created options pricing models only used finite difference methods because the granularity of options pricing models are coarse enough that discrete solutions are usually sufficient, so it was difficult to find comparable articles. One particular article from the Department of Applied Mathematics at the University of Dhaka, Bangladesh decided to perform the same comparison with one finite difference model and one finite element model on the Black-Scholes equation. For their finite difference model, they applied the Du Fort-Frankel finite difference method, which is a slightly different finite difference scheme than the CNM. For their finite element model, they applied GMWR plus CNM to the advection-diffusion version of the Black-Scholes PDE (Shorif Hossan et al., 2020). As a result, the article used the very similar methods compared to this project, different paths to the same destination. The four figures below highlight the results of their paper (Shorif Hossan et al., 2020).

Where the paper also concluded that the results obtained from the finite difference model were better than the results obtained by the finite element model, and that the finite element model tends to deviate from the solution found using equation 12 at certain locations (Shorif Hossan et al., 2020). However, this paper did not suggest any reasons as to why this might have occurred.

The second sanity check supported the Heat FE model as they both concluded that the finite difference model was superior to the finite element model, but it also reaffirms that the Advection-Diffusion FE model was flawed as their paper implemented their FE model using the Advection-Diffusion version of the Black-Scholes PDE with different results than this project.


### 4.1 Conclusion

To determine European option pricing, numerical models for the Black-Scholes PDE were created using finite difference and finite element methods. The resulting models account for both call option pricing and put option pricing because they are distinguished only by their boundary conditions. This report focused mainly on call option pricing. However, Black-Scholes PDE does not account for American options as it does not consider the effects of having the ability to exercise in-the-money options prior to maturity.

Three options pricing models were created and compared with each other and compared with a theoretical Black-Scholes solution as shown in equation 12. One model using strictly finite difference methods with central differencing in the stock price dimension, and Crank-Nicolson Method in the time dimension. The other two models were created using a combination of finite element and finite difference methods, where Galerkin Method of Weighted Residuals was applied in the stock price dimension and Crank-Nicolson Method was applied in the time dimension. The difference between the two finite element models were that one was created using the classical Black-Scholes PDE, resembling an Advection-Diffusion PDE, and the other was created using a transformed Black-Scholes PDE with transformed boundary conditions, resembling a Heat/Diffusion PDE.

It was concluded that the finite element model created using the Advection-Diffusion PDE was not flawed in its derivation logic but flawed with potential human errors or typos either in the code or the derivation, and the flaw(s) have yet to be found. For some unknown reason, the finite difference model was more accurate than the Heat PDE finite element model when compared to equation 12. This apparent relationship was consistent with another article from the University of Dhaka that also attempted a very similar comparison. As a result, it was concluded that the finite difference model and the finite element model created using the Heat PDE were implemented correctly despite the oddity in the finite element model being less accurate than the finite difference model.

Since the finite difference model is simpler to derive, simpler to implement onto a program, requires less computation time to run, and is more accurate than the finite element model using GMWR and Crank-Nicolson Method, a finite difference Black-Scholes model is recommended over a finite element Black-Scholes model in practical applications.

## References

quantpie. (2019). Transformation of Black Scholes PDE to Heat Equation. YouTube. Retrieved April 6, 2022, from https://www.youtube.com/watch?v=IgMoOcO095U.

Seth, S. (2022, March 14). Circumventing the limitations of Black-Scholes. Investopedia.

Retrieved April 6, 2022, from https://www.investopedia.com/articles/active-

trading/041015/how-circumvent-limitations-blackscholes-model.asp

Shorif Hossan, M., Hossain, A. B., \& Shafiqul Islam, M. (2020). Numerical Solutions of BlackScholes model by Du Fort-Frankel FDM and Galerkin WRM. International Journal of Mathematical Research, 9(1), 1-10. https://doi.org/10.18488/journal.24.2020.91.1.10